In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
df_trash = pd.read_csv('../data/trash_hauler_report_with_lat_lng.csv')

In [3]:
# Explore the data.
df_trash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20226 entries, 0 to 20225
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Request Number    20226 non-null  int64  
 1   Date Opened       20226 non-null  object 
 2   Request           20226 non-null  object 
 3   Description       20195 non-null  object 
 4   Incident Address  20217 non-null  object 
 5   Zip Code          20151 non-null  float64
 6   Trash Hauler      19325 non-null  object 
 7   Trash Route       19279 non-null  object 
 8   Council District  20177 non-null  float64
 9   State Plan X      20198 non-null  float64
 10  State Plan Y      20198 non-null  float64
 11  LONGITUDE         20198 non-null  float64
 12  LATITUDE          20198 non-null  float64
dtypes: float64(6), int64(1), object(6)
memory usage: 2.0+ MB


In [4]:
# See a list of all "request" types.
unique_requests = df_trash["Request"].unique()
print(unique_requests)

['Trash - Backdoor' 'Trash - Curbside/Alley Missed Pickup'
 'Trash Collection Complaint' 'Damage to Property']


In [5]:
request_category_counts = df_trash["Request"].value_counts()
print(request_category_counts)

Request
Trash - Curbside/Alley Missed Pickup    15028
Trash - Backdoor                         2629
Trash Collection Complaint               2312
Damage to Property                        257
Name: count, dtype: int64


In [6]:
# Rename columns for uniformity.
df_trash.columns = [col.lower().replace(" ", "_") for col in df_trash.columns]

In [7]:
df_trash.head()

,request_number,date_opened,request,description,incident_address,zip_code,trash_hauler,trash_route,council_district,state_plan_x,state_plan_y,longitude,latitude
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1727970.412,686779.4781,-86.815392,36.217292
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,685444.7996,-86.838103,36.213470
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,659887.4716,-86.885562,36.142923
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1735691.771,685027.2459,-86.789170,36.212652
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,664205.1011,-86.874995,36.154861


In [8]:
df_trash.tail()

,request_number,date_opened,request,description,incident_address,zip_code,trash_hauler,trash_route,council_district,state_plan_x,state_plan_y,longitude,latitude
20221,267125,11/1/2019,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,2731 Murfreesboro Pike,37013.0,RED RIVER,4502,32.0,1781137.263,632448.5511,-86.633970,36.069130
20222,267126,11/1/2019,Trash - Curbside/Alley Missed Pickup,entire alley,"1621 Long Ave, Nashville, TN 37206, United States",37206.0,METRO,9508,6.0,1749711.399,669201.6016,-86.741242,36.169482
20223,267130,11/1/2019,Trash - Curbside/Alley Missed Pickup,missed several,"2943 Windemere Cir, Nashville, TN 37214, Unite...",37214.0,RED RIVER,1502,15.0,1770293.388,674936.3038,-86.671647,36.185643
20224,267134,11/1/2019,Trash - Curbside/Alley Missed Pickup,Caller stated trash was missed & were only pic...,"3325 Murfreesboro Pike, Nashville, TN 37013, U...",37013.0,RED RIVER,4502,32.0,1785224.998,627146.4002,-86.620025,36.054637
20225,267137,11/1/2019,Trash - Curbside/Alley Missed Pickup,possibly others missed as well,"604 Somerset Ct, Nashville, TN 37217, United S...",37217.0,RED RIVER,2505,29.0,1781360.323,637742.0068,-86.633331,36.083675


Missed Trash Pickups

In this data question you will be working data of service request related to missed trash pickups from hubNashville, 
Metro Nashville government's comprehensive customer service system (https://hub.nashville.gov).

As part of Metro's contract with Red River Waste Solutions, failure to remedy an action or inaction will result in liquidated damages. 
One category of liquidated damages is related to chronic problems in any category of service at the same premises. 
A chronic problem is defined as more than one missed pickup for any address. 
The first missed pickup will not result in a fine; however, every subsequent missed pickup will result in a $200 fine.

Your job is to determine the total amount of damages due to missed pickups. Note that not all rows that you have been provided 
correspond to missed pickups and that you will need to ensure that you are only counting missed pickups.

After determining the total amount of damages, you can look at other questions:
⦁	What other types of complaints are there?
⦁	Are there any geospatial analysis you can do?  Which visualizations can you create?
⦁	How do metro crews compare to the contractor's performance?
⦁	How much does each trash hauler owe?
⦁	What were to total missed pickup by route?

Steps to follow:
1. Find all missed pick ups-- Look for "miss" in descriptions for all request categories.
2. Match missed pick ups to addresses.
3. Count occurances at each address.
4. Eliminate first occurance.
5. Multiply by $200 to find fine total.
6. Subset by trash_haou

In [9]:
# Description clean up for uniformity and ease use.
df_trash["description"] = (
    df_trash["description"]
    .str.replace(".", " ", regex=False)
    .str.replace(",", " ", regex=False)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
    .str.lower()
)

# My first version
# df_trash["description"] = df_trash["description"].str.lower()
# df_trash["description"] = df_trash["description"].str.replace(".", " ")
# df_trash["description"] = df_trash["description"].str.strip()

In [10]:
# Find all missed pick ups.
# .str.contains() searches for text patterns in a pandas series.

missed = df_trash['description'].str.contains('missed', case=False, na=False)
print(f"Found {missed.sum()} rows with 'missed'")
df_trash[missed][['description']].head(10)

Found 10785 rows with 'missed'


,description
3,missed
4,missed the even side of the road
8,missed
9,trash missed tuesday again alley
10,missed trash pickup - resident has at curb for...
11,missed- 4th week in a row
12,missed does not have access to alley
14,missed
15,pickup missed
19,missed


In [11]:
# Correct for exact match of "missed".
# Use \b (boundaries) for exact word matches.
# \bmissed\b returns the pattern 'missed' with boundaries on both sides.


missed_exact = df_trash['description'].str.contains(r'\bmissed\b', case=False, na=False, regex=True)
print(f"Found {missed_exact.sum()} rows with exact word 'missed'")
df_trash[missed_exact][['description']].head(10)

Found 10781 rows with exact word 'missed'


,description
3,missed
4,missed the even side of the road
8,missed
9,trash missed tuesday again alley
10,missed trash pickup - resident has at curb for...
11,missed- 4th week in a row
12,missed does not have access to alley
14,missed
15,pickup missed
19,missed


In [12]:
# OR patterns (alternation)
# missed|pickup|alley uses | to mean OR
# regex=True is needed because | is a regex operator meaning OR

keywords = df_trash['description'].str.contains(r'missed|pickup|alley', case=False, na=False, regex=True)
print(f"Found {keywords.sum()} rows with any of these words")
df_trash[keywords][['description']].head(10)

Found 12414 rows with any of these words


,description
3,missed
4,missed the even side of the road
5,left trash cart in middle of driveway instead ...
7,trash out on time miss again tuesday alley
8,missed
9,trash missed tuesday again alley
10,missed trash pickup - resident has at curb for...
11,missed- 4th week in a row
12,missed does not have access to alley
14,missed


In [13]:
# Extract 5 digit zip code from description.
# \d = any digit 0-9
# \d{5} means "exactly 5 digits"

df_trash['extracted_zip'] = df_trash['description'].str.extract(r'(\d{5})')
df_trash[df_trash['extracted_zip'].notna()][['description', 'extracted_zip']].head(10)

,description,extracted_zip
79,trash missed last thursday does not understand...,23455
82,not putting carts back where they got them lid...,90480
147,the whole court miss trash/friday also roundwo...,37013
190,my trash not collected but neighbor's trash wa...,37206
202,trash was not picked up at 244 ash grove dr 37211,37211
276,trash picked up this morning and driver droppe...,37211
363,2867 creekbend drive - 37207 - my trash not pi...,37207
372,trash was not picked up last thursday making s...,37204
404,ref 298388 / cw 939492 is not satisfied with r...,29838
444,none of the trash was not picked up on wednesd...,37215


In [14]:
# Extract 5+4 digit zip code from description.
# \d{5}(?:-\d{4})?
# (?:...) = non-capturing group
# ? = optional (0 or 1 times)

df_trash['zip_plus4'] = df_trash['description'].str.extract(r'(\d{5}(?:-\d{4})?)')
df_trash[df_trash['zip_plus4'].notna()][['description', 'zip_plus4']].head(10)

,description,zip_plus4
79,trash missed last thursday does not understand...,23455
82,not putting carts back where they got them lid...,90480
147,the whole court miss trash/friday also roundwo...,37013
190,my trash not collected but neighbor's trash wa...,37206
202,trash was not picked up at 244 ash grove dr 37211,37211
276,trash picked up this morning and driver droppe...,37211
363,2867 creekbend drive - 37207 - my trash not pi...,37207
372,trash was not picked up last thursday making s...,37204
404,ref 298388 / cw 939492 is not satisfied with r...,29838
444,none of the trash was not picked up on wednesd...,37215


In [15]:
# Extract phone numbers in both formats - xxx-xxx-xxxx/(xxx) xxx-xxxx
# \(?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}
# \(? = optional opening parenthesis
#            [s.-]? = optional space, dot, or dash

df_trash['phone'] = df_trash['description'].str.extract(r'(\(?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4})')
df_trash[df_trash['phone'].notna()][['description', 'phone']].head(10)

,description,phone
0,house with the wheel chair ramp they share dri...,615-876-6274
13,trash truck took cart 3 guys on truck did not ...,(615) 569-4994
39,alley trash miss for wednesday wants cart empt...,615-293-1573
63,missed trash pickup said has been picking up b...,(615) 333-0065
64,missed again picked up everyone except her doe...,(615) 753-1696
79,trash missed last thursday does not understand...,513-403-0759
80,missed pick up - please call (615) 519-3192 wa...,(615) 519-3192
82,not putting carts back where they got them lid...,615-975-1015
139,backdoor trash keeps being missed truck comes ...,615-982-6915
171,curb/trash miss for pick up tuesday out monday...,615-227-9888


In [16]:
# Extract street address like "1234 Main St" or "1234 Main Street"
# \d{1,5}\s+[A-Za-z\s]+(?:St|Dr|Ave|Rd|Pike|Blvd|Ct|Ln)
# \d{1,5} = house numbers with 1 to 5 digits
#        \s+ = one or more spaces
#           [A-Za-z\s]+ = street name (letters and spaces)
#                      (?:St|Dr||Ave|Rd|... = street types

address_pattern = r'\d{1,5}\s+[A-Za-z\s]+(?:st|dr|ave|rd|pike|blvd|ct|ln|way|pl)'
df_trash['address_extracted'] = df_trash['description'].str.extract(f'({address_pattern})')
df_trash[df_trash['address_extracted'].notna()][['description', 'address_extracted']].head(10)

,description,address_extracted
0,house with the wheel chair ramp they share dri...,3817 crouch dr near nes light post
10,missed trash pickup - resident has at curb for...,2017 is st
13,trash truck took cart 3 guys on truck did not ...,3 guys on truck did not have truck number pl
33,5th time in as many weeks metro has not picked...,3702 hobbs rd
61,resident is very upset- red river emptied only...,2 brown carts he had out truck dr
75,trash container out night before pickup in sam...,25 years trash was not picked up as scheduled ...
82,not putting carts back where they got them lid...,2 compl
97,while cleaning the alley behind 1619 heiman st...,1619 heiman st someone from the crew had a wee...
103,while cleaning the alley behind 1619 heiman st...,1619 heiman st someone from the crew had a wee...
124,missed pickup in alley behind 4611 alabama ave,4611 alabama ave


In [17]:
# Find which day pickups were missed.
# (monday|tuesday|...)

days_pattern = r'(monday|tuesday|wednesday|thursday|friday|saturday|sunday|mon|tue|wed|thur|thurs|fri|sat|sun)'
df_trash['day_mentioned'] = df_trash['description'].str.extract(days_pattern, flags=re.IGNORECASE)
df_trash[df_trash['day_mentioned'].notna()][['description', 'day_mentioned']].head(10)

,description,day_mentioned
1,curb/trash miss tuesday,tuesday
2,curb/trash miss tuesday,tuesday
6,trash/emptied wednesday & now metal black-mail...,wednesday
7,trash out on time miss again tuesday alley,tuesday
9,trash missed tuesday again alley,tuesday
18,curb/trash miss for this address and others tu...,tuesday
21,our trash was not picked up today wed nov 1 it...,wed
22,alley trash miss tuesday also others as well o...,tuesday
38,trash missed wednesday curb,wednesday
39,alley trash miss for wednesday wants cart empt...,wednesday


In [18]:
# Find truck numbers like 4887AH (digits followed by letters)
# \d{4}[A-Z]{2}
#      [A-Z] = any upper case letter
#           {2} = exactly 2 times

df_trash['truck_number'] = df_trash['description'].str.extract(r'(\d{4}[a-z]{2})')
df_trash[df_trash['truck_number'].notna()][['description', 'truck_number']].head()

,description,truck_number
20,a resident is complaining that a trash truck r...,4887ah
1805,there was a public works trash truck driving d...,4908ag
5941,i need a 105 property damaged filled out pleas...,5219ah
6402,brush truck # 4249ac pulled an at&t line from ...,4249ac
10377,driver almost caused a wreck bumper#5123ah,5123ah


In [19]:
# Check if the description starts with a number.
# ^\d+
# ^ = start of a string
# $ = end of a string
# + = one or more times

# Use .str.contains() with ^ anchor instead of .str.match() to avoid NaN issues.

starts_with_number = df_trash['description'].str.contains(r'^\d+', na=False)
print(f"descriptions starting with a number: {starts_with_number.sum()}")
df_trash[starts_with_number][['description']].head()

descriptions starting with a number: 773


,description
33,5th time in as many weeks metro has not picked...
227,8 houses missed last thursday every other week...
346,626 40th ave n - clifton ave market- 4 trash c...
350,3554 gondola dr missed one side of the road
359,2 trash/carts curb full carts needs to be empt...


In [20]:
# Tag rows by complaint type keywords.

df_trash['is_missing_pickup'] = df_trash['request'].str.contains(r'missed pickup', case=False, na=False)
df_trash['is_damage'] = df_trash['request'].str.contains(r'damage', case=False, na=False)
df_trash['is_backdoor'] = df_trash['request'].str.contains(r'backdoor', case=False, na=False)

# Count each type
print(f"Missed Pickups: {df_trash['is_missing_pickup'].sum()}")
print(f"Damage Complaints: {df_trash['is_damage'].sum()}")
print(f"Backdoor Requests: {df_trash['is_backdoor'].sum()}")

Missed Pickups: 15028
Damage Complaints: 257
Backdoor Requests: 2629


In [21]:
# Find unique request numbers mentioned in the 'description' column.
# Extract 5 digit numbers that might be request references.

request_refs = df_trash['description'].str.findall(r'\b\d{5}\b')

# Count how many descriptions reference other requests.
# Check NaN values by seeing if x is a list first.

has_reference = request_refs.apply(lambda x: len(x) > 0 if isinstance(x, list) else False)
print(f"Descriptions mentioning 5-digit numbers: {has_reference.sum()}")
df_trash[has_reference][['request_number', 'description']].head()

Descriptions mentioning 5-digit numbers: 186


,request_number,description
79,25862,trash missed last thursday does not understand...
147,26391,the whole court miss trash/friday also roundwo...
190,27220,my trash not collected but neighbor's trash wa...
202,27318,trash was not picked up at 244 ash grove dr 37211
276,28382,trash picked up this morning and driver droppe...


In [22]:
# Normalize text 

df_trash['clean_description'] = (df_trash['description']
    .str.strip()     # removes leading/trailing spaces
    .str.replace(r'\s+', ' ', regex=True)     # collapse multiple spaces
    .str.replace(r'[.|,]+$', '', regex=True)     # remove trailing punctuation
    .str.lower()
                                )

# Compare original vs cleaned

df_trash[['description', 'clean_description']].head(10)

,description,clean_description
0,house with the wheel chair ramp they share dri...,house with the wheel chair ramp they share dri...
1,curb/trash miss tuesday,curb/trash miss tuesday
2,curb/trash miss tuesday,curb/trash miss tuesday
3,missed,missed
4,missed the even side of the road,missed the even side of the road
5,left trash cart in middle of driveway instead ...,left trash cart in middle of driveway instead ...
6,trash/emptied wednesday & now metal black-mail...,trash/emptied wednesday & now metal black-mail...
7,trash out on time miss again tuesday alley,trash out on time miss again tuesday alley
8,missed,missed
9,trash missed tuesday again alley,trash missed tuesday again alley


In [23]:
# Total calls in all request categories except damage.
request_category_missed = df_trash[df_trash["request"].
    isin(["Trash - Backdoor", "Trash - Curbside/Alley Missed Pickup", "Trash Collection Complaint"])].shape[0]
print(request_category_missed)

19969


In [24]:
# Address clean up for uniformity and ease of use. Removing punctuation and extra spaces inside and out.
df_trash["address_clean"] = (
    df_trash["incident_address"]
    .astype(str)
    .str.replace(".", " ", regex=False)
    .str.replace(",", " ", regex=False)
    .str.lower()
    .str.strip()
)

# My first version
# df_trash["incident_address"] = df_trash["incident_address"].str.title()
# df_trash["incident_address"] = df_trash["incident_address"].str.replace(".", " ")
# df_trash["incident_address"] = df_trash["incident_address"].str.strip()

In [25]:
# Remove City, State, and United States from address column.

df_trash["address_clean"] = (
    df_trash["address_clean"]
    .str.replace(r"\bnashville\b", "", regex=True)
    .str.replace(r"\btn\b", "", regex=True)
    .str.replace(r"\bunited states\b", "", regex=True)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

In [30]:
# Abbreviate most common street types.

df_trash["address_clean"] = (
    df_trash["address_clean"]
    .str.replace(r"\bavenue\b", "ave", regex=True)
    .str.replace(r"\bstreet\b", "st", regex=True)
    .str.replace(r"\bdrive\b", "dr", regex=True)
    .str.replace(r"\broad\b", "rd", regex=True)
    .str.replace(r"\bpike\b", "pk", regex=True)
    .str.replace(r"\bboulevard\b", "blvd", regex=True)
    .str.replace(r"\bcourt\b", "ct", regex=True)
    .str.replace(r"\blane\b", "ln", regex=True)
    .str.replace(r"\bplace\b", "pl", regex=True)
    .str.replace(r"\bparkway\b", "pkwy", regex=True)
    .str.replace(r"\btrail\b", "trl", regex=True)
)

In [34]:
# Count requests per address.

address_counts = df_trash["address_clean"].value_counts()
print(address_counts)      

address_clean
5135 hickory hollow pkwy        21
3710 n natchez ct               21
6007 obrien ave 37209           19
12546 old hickory blvd 37013    19
802 crescent rd 37205           18
                                ..
4226 eatons creek rd 37218       1
6726 pennywell dr 37205          1
115 colemont ct 37013            1
103 braxton hill ct 37204        1
2924 fernbrook ln 37214          1
Name: count, Length: 13605, dtype: int64


In [49]:
# Check the total number of missed pickups.

total_missed_pickups = len(df_trash)
total_missed_pickups

total_missed_pickups = df_trash[df_trash["request"].
    isin(["Trash - Backdoor", "Trash - Curbside/Alley Missed Pickup", "Trash Collection Complaint"])].shape[0]
print(total_missed_pickups)

19969


In [35]:
# Make it a dataframe

address_counts_df = (
    df_trash
    .groupby("address_clean")
    .size()
    .reset_index(name="request_count")
    .sort_values("request_count", ascending=False)
)

In [36]:
# Quick check with a popular street to look at the data.

df_trash[df_trash["address_clean"].str.contains("old hickory", na=False)]

,request_number,date_opened,request,description,incident_address,zip_code,trash_hauler,trash_route,council_district,state_plan_x,...,zip_plus4,phone,address_extracted,day_mentioned,truck_number,is_missing_pickup,is_damage,is_backdoor,clean_description,address_clean
20,25465,11/1/2017,Trash Collection Complaint,a resident is complaining that a trash truck r...,1414 Old Hickory Blvd,37207.0,NaN,NaN,3.0,1738736.963,...,NaN,NaN,NaN,NaN,4887ah,False,False,False,a resident is complaining that a trash truck r...,1414 old hickory blvd
90,25951,11/3/2017,Trash - Backdoor,never gets picked up please pick her up has to...,15482 Old Hickory Blvd,37211.0,RED RIVER,4402,27.0,1751222.233,...,NaN,NaN,NaN,thursday,NaN,False,False,True,never gets picked up please pick her up has to...,15482 old hickory blvd
291,28542,11/16/2017,Trash - Backdoor,said never gets picked up - said trash buildin...,15482 Old Hickory Blvd,37211.0,RED RIVER,4402,27.0,1751222.233,...,NaN,NaN,2019 please note if something is not compl,NaN,NaN,False,False,True,said never gets picked up - said trash buildin...,15482 old hickory blvd
717,34020,12/18/2017,Trash - Backdoor,back door pick up said never gets picked up - ...,15482 Old Hickory Blvd,37211.0,RED RIVER,4402,27.0,1751222.233,...,NaN,NaN,NaN,NaN,NaN,False,False,True,back door pick up said never gets picked up - ...,15482 old hickory blvd
1117,38589,1/12/2018,Trash - Curbside/Alley Missed Pickup,curb trash/keeps getting missed thursday pick up,15409 Old Hickory Blvd,37211.0,RED RIVER,4402,4.0,1752557.806,...,NaN,NaN,NaN,thursday,NaN,True,False,False,curb trash/keeps getting missed thursday pick up,15409 old hickory blvd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18796,254568,10/2/2019,Trash - Curbside/Alley Missed Pickup,my entire street (at the very least) was not p...,"320 Old Hickory Blvd, Nashville, TN 37221, Uni...",37221.0,RED RIVER,1303,23.0,1697705.577,...,NaN,NaN,NaN,NaN,NaN,True,False,False,my entire street (at the very least) was not p...,320 old hickory blvd 37221
19169,258226,10/11/2019,Trash - Curbside/Alley Missed Pickup,still out,"309 W Old Hickory Blvd, Madison, TN 37115, Uni...",37115.0,RED RIVER,3214,8.0,1757078.005,...,NaN,NaN,NaN,NaN,NaN,True,False,False,still out,309 w old hickory blvd madison 37115
19642,261137,10/18/2019,Trash - Curbside/Alley Missed Pickup,no one was pickup on ohb curb thursday,"15459 Old Hickory Blvd, Nashville, TN 37211, U...",37211.0,RED RIVER,4402,4.0,1751053.588,...,NaN,NaN,NaN,thursday,NaN,True,False,False,no one was pickup on ohb curb thursday,15459 old hickory blvd 37211
20148,266412,10/31/2019,Trash - Curbside/Alley Missed Pickup,trash was not picked up for 2 weeks in row,"845 Old Hickory Blvd, Madison, Tennessee, 37115",37115.0,RED RIVER,3212,8.0,1748845.000,...,NaN,NaN,NaN,NaN,NaN,True,False,False,trash was not picked up for 2 weeks in row,845 old hickory blvd madison tennessee 37115


In [39]:
# Pair the address with the count of complaints.

address_counts = (
    df_trash
    .groupby("address_clean")
    .size()
    .reset_index(name="complaint_count")
)
print(address_counts)

                    address_clean  complaint_count
0                                                1
1           1 belle forrest ave c                1
2            10 belle forrest ave                1
3      100 bluefield square 37214                1
4             100 braxton hill ct                1
...                           ...              ...
13600            trevino pl 37013                2
13601   trimble rd & lindawood dr                1
13602                 tusculum rd                3
13603           westboro dr 37209                1
13604                   xavier dr                1

[13605 rows x 2 columns]


In [41]:
# Subtract 1 from the per address counts to find the number of fines per address.

address_counts["fines"] = address_counts["complaint_count"] - 1
address_counts["fines"] = address_counts["fines"].clip(lower=0)
print(address_counts)

                    address_clean  complaint_count  fines
0                                                1      0
1           1 belle forrest ave c                1      0
2            10 belle forrest ave                1      0
3      100 bluefield square 37214                1      0
4             100 braxton hill ct                1      0
...                           ...              ...    ...
13600            trevino pl 37013                2      1
13601   trimble rd & lindawood dr                1      0
13602                 tusculum rd                3      2
13603           westboro dr 37209                1      0
13604                   xavier dr                1      0

[13605 rows x 3 columns]


In [43]:
df_trash = df_trash.merge(
    address_counts[["address_clean", "fines"]],
    on="address_clean",
    how="left"
)
df_trash.head()

,request_number,date_opened,request,description,incident_address,zip_code,trash_hauler,trash_route,council_district,state_plan_x,...,phone,address_extracted,day_mentioned,truck_number,is_missing_pickup,is_damage,is_backdoor,clean_description,address_clean,fines
0,25270,11/1/2017,Trash - Backdoor,house with the wheel chair ramp they share dri...,3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1727970.412,...,615-876-6274,3817 crouch dr near nes light post,NaN,NaN,False,False,True,house with the wheel chair ramp they share dri...,3817 crouch dr,0
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,curb/trash miss tuesday,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1721259.366,...,NaN,NaN,tuesday,NaN,True,False,False,curb/trash miss tuesday,4028 clarksville pk,1
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,curb/trash miss tuesday,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1707026.753,...,NaN,NaN,tuesday,NaN,True,False,False,curb/trash miss tuesday,6528 thunderbird dr,0
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1735691.771,...,NaN,NaN,NaN,NaN,True,False,False,missed,2603 old matthews rd,0
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed the even side of the road,604 croley dr,37209.0,RED RIVER,4203,20.0,1710185.772,...,NaN,NaN,NaN,NaN,True,False,False,missed the even side of the road,604 croley dr,0


In [51]:
# Calculate the number of fines for each trash hauler.

hauler_fines = (
    df_trash
    .drop_duplicates("address_clean")
    .groupby("trash_hauler")["fines"]
    .sum()
    .reset_index(name="total_fines")
    .sort_values("total_fines", ascending=False)
)
print(hauler_fines)

  trash_hauler  total_fines
2    RED RIVER         4877
0        METRO         1148
3    WASTE IND          407
1        Metro           30


In [58]:
# Calculate the total fine amount for each hauler.

df_trash["fine_amount"] = df_trash["fines"] * 200
total_fine_amount = (
    df_trash
    .drop_duplicates("address_clean")
    ["fine_amount"]
    .sum()
)
print(total_fine_amount)

1324200


In [60]:
# Find Red River's fines.

red_river_total_fines = (
    df_trash
    .drop_duplicates("address_clean")
    .loc[df_trash["trash_hauler"].str.contains("red river", case=False, na=False), "fine_amount"]
    .sum()
)
print(red_river_total_fines)

975400


In [47]:
# Verify the address that made me question everything I did earlier...

address_counts[address_counts["address_clean"] == "14885 old hickory blvd"]

,address_clean,complaint_count,fines
2055,14885 old hickory blvd,10,9


In [ ]:
# Multiply the fines by $200



# Return NaN values unchanged.
def keep_street_only(address):
    if pd.isna(address):
        return address
# This line unintentionally removed 5 digit street addresses.
    # re.sub(r"\b\d{5}(?:-\d{4})?\b", "", address)
# Remove 5 or 9 digit strings IF they follow a 2 character set (hopefully, this only captures zip codes)
    address = re.sub(r"(,?\s+[A-Za-z]{2}\s*)\d{5}(?:-\d{4})?$", r"\1", address)
# Remove "United States"
    address = re.sub(r"\bUnited States\b", "", address)   
# Remove city "Nashville" using the key that it will be followed by a state abbreviation.
# This is in case there is a street with "Nashville" in the name.
    address = re.sub(r"\bNashville\b(?=\s+[A-Z]{2}\b)", "", address, flags=re.IGNORECASE)   
# Remove "Tn"
    address = re.sub(r"\bTn\b", "", address)
# Remove blank spaces.  
    address = " ".join(address.split())
    return address

# Assign the cleaned address column to a new dataframe.
df_trash_clean_address = df_trash.copy()

df_trash_clean_address["incident_address"] = (
    df_trash_clean_address["incident_address"]
    .apply(keep_street_only)
)

# IT WORKED!
df_trash_clean_address.tail()

# To compare the before and after result. \(^O^)/  <--Much rejoicing!
df_trash.tail(5)

In [26]:
# Create a df with the three "request" categories that contain missed pick ups.
df_missed_list_all = df_trash_clean_address[df_trash_clean_address["request"]
    .isin(["Trash - Backdoor", "Trash - Curbside/Alley Missed Pickup", "Trash Collection Complaint"])].copy()
df_missed_list_all.info()

NameError: name 'df_trash_clean_address' is not defined

In [ ]:
address_counts = df_missed_list_all["incident_address"].value_counts()
print(address_counts)

In [ ]:
# SOMETHING IS WAAAY WRONG. THIS ADDRESS APPEARS MANY TIMES IN THE CSV
address_to_check = "14885 Old Hickory Blvd"
count = counts.get(address_to_check, 0)
print(f"{address_to_check} appears {count} times")

In [ ]:
address_to_check = "14885 Old Hickory Blvd"

matching_rows = df_missed_list_all[
    df_missed_list_all["incident_address"].str.contains(address_to_check, case=False, na=False)
]

print(matching_rows)
print(f"Number of matches: {matching_rows.shape[0]}")

In [ ]:
# search df_trash for 14885 Old Hickory

In [ ]:
# See a list of all "Missed Pickup" requests with "descriptions" that contain the word "miss".
request_trash_missed = df_trash[df_trash["request"] == "Trash - Curbside/Alley Missed Pickup"]
unique_trash_missed = request_trash_missed[request_trash_missed["description"].str.contains("miss", case=False, na=False)]
unique_miss_entries = unique_trash_missed["description"].unique()
print(unique_miss_entries)

In [ ]:
# See a list of all descriptions that contain "miss" across all request types.
miss_entries = df_trash[df_trash["description"].str.contains("miss", case=False, na=False)]
unique_miss_entries = miss_entries["description"].unique().tolist()
print(unique_miss_entries)

In [ ]:
# There are 7007 unique descriptions across all requests that contain the word "miss".
unique_miss_entries_count = len(set(unique_miss_entries))
print(unique_miss_entries_count)

In [ ]:
trash_missed_df = df_trash[df_trash['description'].str.contains('missed| miss', case=False, na=True)]
trash_missed_df.info()
trash_missed_df.head()

In [ ]:
# How many contractors pick up trash?
unique_contractors = df_trash["trash_hauler"].unique()
print(unique_contractors)

In [ ]:
# Create df with only RED RIVER.
red_river_df = df_trash[df_trash["trash_hauler"] == "RED RIVER"]

In [ ]:
# Reset the index of red_river_df 
red_river_df = red_river_df.reset_index(drop=True)

In [ ]:
# Filter red_river_df for "descriptions" that contain "miss".
red_river_missed = red_river_df[red_river_df['description'].str.contains('missed| miss', case=False, na=True)]
red_river_missed.info()
red_river_missed.head()

BONUS!

Missed Trash Pickups
The city is considering different ways to calculate fines. Here are two methods that are being considered. 
Calculate the total fines that would be collected by each and compare that total to the amount collected using the current method.

Alternative Method 1:
For each address, if there are three or more missed pickups within a 180-day period, damages of $1500 will be charged. 
(A fine will be levied every time three unique missed pickup dates occur within a six-month period for a single address.)

Alternative Method 2:
This method also considers the six-month window like Alternative Method 1, but each date can only be used once to support a fine. 
How will this difference impact the fines levied? (Example: If Jan 1st, Mar 3rd, Apr 8th, and Aug 9th were the only four dates a 
trash pickup was missed, the original method would result in $3000 in fines [Jan, Mar, Apr, and also Mar, Apr, Aug]. 
However, this updated method would only result in $1500 because neither Mar nor Apr can be used for another fine since they were already used.)
